In [1]:
import os
import requests
import json
import time
import tweepy
import sys
import mysql.connector
from mysql.connector import Error
from dateutil import parser
from pprint import pprint
from requests.auth import AuthBase
from requests.auth import HTTPBasicAuth


In [2]:
consumer_key = "XPxx7q10newFt0aBAmUbgz23S"  # Add your API key here
consumer_secret = "GUljnxIUhuE2svMrbzfV7uQBsBRv2r6VVU0GPZ53zJloqFyn8k"  # Add your API secret key here

In [3]:
access_token ='1306748755220131840-hcPz6ZW6y2q9wzOslctZFljG86uhIE'
access_secret ='vIWdMwQJ02OptOUJDQmXyRAvOLGT6Vgq7dbraiHJpS9Hf'

In [4]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if hasattr(status, "retweeted_status"): #Check if a retweet
            try:
                current_status = str(status.retweeted_status.extended_tweet["full_text"])
                current_status = current_status.replace('\n', '').replace('\r', '')
                file1.write(current_status)
                file1.write('\n')
                print(current_status)
            except AttributeError:
                current_status = str(status.retweeted_status.text)
                current_status = current_status.replace('\n', '').replace('\r', '')
                file1.write(current_status)
                file1.write('\n')
                print(current_status)
        else:
            try:
                current_status = str(status.extended_tweet["full_text"])
                current_status = current_status.replace('\n', '').replace('\r', '')
                file1.write(current_status)
                file1.write('\n')
                print(current_status)
            except AttributeError:
                current_status = str(status.text)
                current_status = current_status.replace('\n', '').replace('\r', '')
                file1.write(current_status)
                file1.write('\n')
                print(current_status)
                
    def on_error(self,status_code):
        if status_code == 420:
            print('You have been rate limited for too many requests.')
            return False #ends the session
        elif status_code != 200:
            print('error code not 200')
            return False
        
    def on_data(self,data):
        try:
            raw_data=json.loads(data)
            if 'text' in raw_data:
                username = raw_data['user']['screen_name']
                created_at = parser.parse(raw_data['created_at'])
                tweet = raw_data['text']
                
                if raw_data['place'] is not None:
                    place = raw_data['place']['country']
                    print(place)
                else:
                    place = None
                    
                location = raw_data['user']['location']
                #change this to the ID_STR for the unique ID of the Tweet
                id_str = raw_data['id_str']
                
                connect(username,created_at,tweet, place, location, id_str)
                print("Tweet collected at: {}".format(str(created_at)))
                
        except Error as e:
            print('error extracting tweet on_data method with Json')
            print(e)

In [5]:
def connect(username, created_at, tweet, place , location, id_str):
    """
    connect to Database, insert the data
    """
    
    try:
        conn = mysql.connector.connect(host='localhost',database = 'twitterDB',user= 'nickrolph', password = 'password', charset = 'utf8')
        
        if conn.is_connected():
            """
            Insert data from stream
            """
            
            cursor = conn.cursor()
            query = "INSERT INTO LostAndFoundPets (username, created_at, tweet, place, location, id_str) VALUES (%s, %s, %s, %s, %s, %s)"
            print("attempting to insert")
            cursor.execute(query, (username, created_at, tweet, place, location, id_str))
            conn.commit()
    
    except Error as e:
        print(e)
        
    cursor.close()
    conn.close()
    
    return

Main method begins here at cell 15

In [6]:
auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)
api = tweepy.API(auth) #make API object

In [7]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth,listener=myStreamListener)

In [8]:
myStream.filter(track = ["#lostdog","#founddog","#lostcat","#foundcat","#lostpet","#foundpet"])

attempting to insert
Tweet collected at: 2020-10-21 06:40:52+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:50:48+00:00
attempting to insert
1366 (HY000): Incorrect string value: '\xF0\x9F\x93\x9E03...' for column 'tweet' at row 1
Tweet collected at: 2020-10-21 06:53:29+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:54:36+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:55:49+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:58:30+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:58:35+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:58:41+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:59:04+00:00
attempting to insert
Tweet collected at: 2020-10-21 06:59:32+00:00
attempting to insert
Tweet collected at: 2020-10-21 07:00:02+00:00
attempting to insert
Tweet collected at: 2020-10-21 07:01:16+00:00
attempting to insert
1366 (HY000): Incorrect string value: '\xF0\x9F\x99\x8F\xF0\x9F...' for column 'tweet

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.